In [11]:
import pandas as pd
import numpy as np
import csv

'''
let A = the first input table (or ideally, the larger one)
let B = the second input table (or ideally, the smaller one)
let jA = the join column ID of table A
let jB = the join column ID of table B
let MB = a multimap for mapping from single values to multiple rows of table B (starts out empty)
let C = the output table (starts out empty)

for each row b in table B:
   place b in multimap MB under key b(jB)

for each row a in table A:
   for each row b in multimap MB under key a(jA):
      let c = the concatenation of row a and row b
      place row c in table C
'''

TableR = pd.read_csv("TableR.csv")
TableS = pd.read_csv("TableS.csv")



print(TableR)
print(TableS)


   ID     Name
0   1     Jere
1   2     Olga
2   3  Timothy
3   3    Anton
   ID    LastName
0   1      Huggel
1   2   Bespalova
2   3  Zimmermann


In [20]:
#HashJoin
TableT = {}

for person in TableS["ID"]:
    if person not in TableT:
        TableT[person] = [TableS[TableS["ID"] == person].values.flatten().tolist()]
    else:
        TableT[person] = TableT[person].append( TableS[TableS["ID"] == person].values.flatten().tolist())
#(s, r) for r in table2 for s in h[r[index2]]

for index, R in TableR.iterrows():
    for S in TableT[R[0]]:
        S.append(R.values.flatten().tolist())
        
print(TableT)

{1: [[1, 'Huggel', [1, 'Jere']]], 2: [[2, 'Bespalova', [2, 'Olga']]], 3: [[3, 'Zimmermann', [3, 'Timothy'], [3, 'Anton']]]}


In [19]:
# 2-phase Track-Join
# Initialize Table Tr|s

TableTrs = []
sent_to_R = []
sent_to_S = []

def send_to_t(index, table):
    print('%s sent %d to T' %(table, index))
    TableTrs.append([index, table])
    
def send_to_s(index, payload):
    print('%s sent %d to S' %(payload.values.flatten().tolist(), index))
    sent_to_S.append([index, payload])

def send_to_r(index, table):
    print('%s sent %d to R' %(table, index))
    sent_to_R.append([index, table])

# ->ProcessR

print(' ---> Process R ')

TableTr = {}

for index, person in TableR.iterrows():
    if person['ID'] not in TableTr:
        send_to_t(person['ID'], 'R')
        TableTr[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTr[person['ID']] = [TableTr[person['ID']], person.values.flatten().tolist()]


print(TableTr)
print(TableTrs)
print('---Barrier ProcessR----- \n ---> Process S ')

# ->ProcessS

TableTs = {}

for index, person in TableS.iterrows():
    if person['ID'] not in TableTs:
        send_to_t(person['ID'], 'S')
        TableTs[person['ID']] = [person.values.flatten().tolist()]
    else:
        TableTs[person['ID']] = [TableTs[person['ID']], person.values.flatten().tolist()]


print(TableTs)
print(TableTrs)
print('---Barrier Process S----- \n ---> Process T resumes')

# ->ProcessT after barrier

TableTrs_cpy = TableTrs
for row in TableTrs:
    if row[1] == 'R':
        for row2 in TableTrs_cpy:
            if row2[1] == 'S' and row[0] == row2[0]:
                send_to_r(row2[0], 'T')
print(sent_to_R)
print('---Ended Process T----- \n ---> Process R resumes')

# ->ProcessR after barrier

for row in sent_to_R:
    for index, row_val in TableR[TableR['ID'] == row[0]].iterrows():
        send_to_s(row[0], row_val)

print('---Ended Process R----- \n ---> Process S resumes')

# ->ProcessS after barrier

for row in sent_to_S:
    for index, row_val in TableS[TableS['ID'] == row[0]].iterrows():
        print(str(row_val.values.flatten().tolist()) + str(row[1].values.flatten().tolist()))
print('---Ended Process S-----')

 ---> Process R 
R sent 1 to T
R sent 2 to T
R sent 3 to T
{1: [[1, 'Jere']], 2: [[2, 'Olga']], 3: [[[3, 'Timothy']], [3, 'Anton']]}
[[1, 'R'], [2, 'R'], [3, 'R']]
---Barrier ProcessR----- 
 ---> Process S 
S sent 1 to T
S sent 2 to T
S sent 3 to T
{1: [[1, 'Huggel']], 2: [[2, 'Bespalova']], 3: [[3, 'Zimmermann']]}
[[1, 'R'], [2, 'R'], [3, 'R'], [1, 'S'], [2, 'S'], [3, 'S']]
---Barrier Process S----- 
 ---> Process T resumes
T sent 1 to R
T sent 2 to R
T sent 3 to R
[[1, 'T'], [2, 'T'], [3, 'T']]
---Ended Process T----- 
 ---> Process R resumes
[1, 'Jere'] sent 1 to S
[2, 'Olga'] sent 2 to S
[3, 'Timothy'] sent 3 to S
[3, 'Anton'] sent 3 to S
---Ended Process R----- 
 ---> Process S resumes
[1, 'Huggel'][1, 'Jere']
[2, 'Bespalova'][2, 'Olga']
[3, 'Zimmermann'][3, 'Timothy']
[3, 'Zimmermann'][3, 'Anton']
---Ended Process S-----
